In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

Tesla T4


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import tqdm
import pickle

# Load the test set
with open("/content/drive/MyDrive/test_set.pkl", "rb") as f:
    testset = pickle.load(f)

total_images = len(testset)
print("Total number of images in the test dataset:", total_images)

batch_size = 64  # Replace XX with your batch size
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Total number of images in the test dataset: 100


In [4]:
# First, upload the file or copy the file into your google drive.
# And mount your drive. Then load the model

model = torch.load('/content/drive/MyDrive/resnet.model', map_location=torch.device('cpu')) # e.g. /content/drive/MyDrive/CS255/resnet.model
# print(model)

model = model.to('cuda:0') # if GPU available
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
def fast_gradient_sign_method(model, input_imgs, labels, epsilon=0.16732, device='cuda:0'):
    input_imgs = input_imgs.to(device)
    labels = labels.to(device)

    input_imgs.requires_grad = True # Ensure that we're tracking gradients for input images
    preds = model(input_imgs) # Forward pass
    loss = F.cross_entropy(preds, labels)    # Calculate the loss
    model.zero_grad() # Zero all existing gradients
    loss.backward() # Backward pass (to compute gradients)
    data_grad = input_imgs.grad.data     # Collect the gradients of the input images
    perturbed_image = input_imgs + epsilon * data_grad.sign()

    # Adding clipping to maintain the original range of the image
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image

In [6]:
def pgd(model, input_imgs, labels, epsilon=0.09, alpha=2/255, num_iter=100, device='cuda:0'):
    input_imgs = input_imgs.to(device)
    labels = labels.to(device)

    delta = torch.zeros_like(input_imgs, requires_grad=True).to(device) # Initialize delta
    alpha_tensor = torch.tensor(alpha, device=device, dtype=input_imgs.dtype) # Convert alpha to a tensor and send it to the correct device

    for t in tqdm.tqdm(range(num_iter), desc="PGD Progress"):
        outputs = model(input_imgs + delta)  # Forward pass
        loss = F.cross_entropy(outputs, labels) # Calculate the loss
        model.zero_grad() # Zero all existing gradients
        loss.backward()  # Backward pass
        delta.data = (delta + alpha_tensor * delta.grad.detach().sign()).clamp(-epsilon, epsilon) # Update delta within the constraints of epsilon and clamp
        delta.grad.zero_() # Zero the gradients for the next iteration

    adversarial_samples = input_imgs + delta.detach() # Create adversarial samples

    return adversarial_samples

In [7]:
def eval_model(model, dataset_loader, img_func=fast_gradient_sign_method):
    tp, tp_5, counter = 0., 0., 0.
    for imgs, labels in tqdm.tqdm(dataset_loader, desc="Validating..."):
        imgs = imgs.to('cuda:0')
        labels = labels.to('cuda:0')
        if img_func is not None:
            imgs = imgs + img_func(model, imgs, labels)
        with torch.no_grad():
            preds = model(imgs)
        tp += (preds.argmax(dim=-1) == labels).sum()
        tp_5 += (preds.topk(5, dim=-1)[1] == labels[...,None]).any(dim=-1).sum()
        counter += preds.shape[0]
    acc = tp.float().item()/counter
    top5 = tp_5.float().item()/counter
    print(f"Top-1 error: {(100.0 * (1 - acc)):4.2f}%")
    print(f"Top-5 error: {(100.0 * (1 - top5)):4.2f}%")
    return acc, top5

## Doing grid Search:-
--> I have used type of grid search to find the epsilon value that reduces the Top-5 Accuracy to be less that 85% such according to self_test function given in the Lab4_document and found that epsilon to be ``0.16732028126716614``.

In [8]:
epsilon_values = torch.linspace(start=0, end=64/255, steps=10)  # Adjust the range and steps based on your needs
top5_accuracies = []

for eps in epsilon_values:
    print(f"Testing with epsilon: {eps}")
    _, top5 = eval_model(model, testloader, lambda m, i, l: fast_gradient_sign_method(m, i, l, epsilon=eps))
    top5_accuracies.append(top5)
    if top5 < 0.85:
        print(f"Top-5 accuracy below 85% achieved with epsilon: {eps}")
        break

Testing with epsilon: 0.0


Validating...: 100%|██████████| 2/2 [00:00<00:00,  2.37it/s]


Top-1 error: 32.00%
Top-5 error: 5.00%
Testing with epsilon: 0.027886711061000824


Validating...: 100%|██████████| 2/2 [00:00<00:00,  8.63it/s]


Top-1 error: 45.00%
Top-5 error: 7.00%
Testing with epsilon: 0.05577342212200165


...

Top-1 error: 71.00%
Top-5 error: 13.00%
Testing with epsilon: 0.13943356275558472


Validating...: 100%|██████████| 2/2 [00:00<00:00,  9.38it/s]


Top-1 error: 76.00%
Top-5 error: 13.00%
Testing with epsilon: 0.16732028126716614


Validating...: 100%|██████████| 2/2 [00:00<00:00,  9.41it/s]

Top-1 error: 80.00%
Top-5 error: 18.00%
Top-5 accuracy below 85% achieved with epsilon: 0.16732028126716614


In [9]:
# Evaluate the model
accuracy, top5_accuracy = eval_model(model, testloader, fast_gradient_sign_method)
print("Accuracy:", accuracy)
print("Top-5 Accuracy:", top5_accuracy)

Validating...: 100%|██████████| 2/2 [00:00<00:00,  9.41it/s]

Top-1 error: 80.00%
Top-5 error: 18.00%
Accuracy: 0.2
Top-5 Accuracy: 0.82


In [10]:
# Evaluate the model
accuracy, top5_accuracy = eval_model(model, testloader, pgd)
print("Accuracy:", accuracy)
print("Top-5 Accuracy:", top5_accuracy)

Validating...: 100%|██████████| 2/2 [00:19<00:00,  9.57s/it]

Top-1 error: 85.00%
Top-5 error: 18.00%
Accuracy: 0.15
Top-5 Accuracy: 0.82


In [11]:
def save_imgs(model, dataset_loader, img_func=fast_gradient_sign_method):
    X_train = []
    Y_train = []
    for imgs, labels in tqdm.tqdm(dataset_loader, desc="saving..."):
        imgs = imgs.to('cuda:0')
        labels_gpu = labels.to('cuda:0')

        # Apply the adversarial attack
        imgs = imgs + img_func(model, imgs, labels_gpu)

        # Iterate over the actual number of images in this batch
        for i in range(imgs.size(0)):
            X_train.append(imgs[i].detach().cpu())
            Y_train.append(labels[i].cpu())  # No need to detach as labels are not part of the computation graph

    # Combine the images and labels into a single dataset
    sampled_test_data = [(X, Y) for X, Y in zip(X_train, Y_train)]

    # Save the dataset as a pickle file
    with open("fgsm.pkl", "wb") as f:
        pickle.dump(sampled_test_data, f)

In [12]:
save_imgs(model, testloader, fast_gradient_sign_method)

saving...: 100%|██████████| 2/2 [00:00<00:00, 11.26it/s]


In [13]:
def save_imgs_pgd(model, dataset_loader, img_func=fast_gradient_sign_method):
    X_train = []
    Y_train = []
    for imgs, labels in tqdm.tqdm(dataset_loader, desc="saving..."):
        imgs = imgs.to('cuda:0')
        labels_gpu = labels.to('cuda:0')

        # Apply the adversarial attack
        imgs = imgs + img_func(model, imgs, labels_gpu)

        # Iterate over the actual number of images in this batch
        for i in range(imgs.size(0)):
            X_train.append(imgs[i].detach().cpu())
            Y_train.append(labels[i].cpu())  # No need to detach as labels are not part of the computation graph

    # Combine the images and labels into a single dataset
    sampled_test_data = [(X, Y) for X, Y in zip(X_train, Y_train)]

    # Save the dataset as a pickle file
    with open("pgd.pkl", "wb") as f:
        pickle.dump(sampled_test_data, f)

In [14]:
save_imgs_pgd(model, testloader, pgd)

saving...: 100%|██████████| 2/2 [00:14<00:00,  7.35s/it]


In [15]:
def self_check(model, result_set='fgsm.pkl'):
    # Load the adversarial and original test set
    with open(result_set, "rb") as f:
        resultset = pickle.load(f)
    with open("/content/drive/MyDrive/test_set.pkl", "rb") as f2:
        testset = pickle.load(f2)

    # Prepare the evaluation set
    eval_set = []
    for org, adv in zip(testset, resultset):
        assert org[1] == adv[1]  # Make sure labels match
        eval_set.append((org[0], adv[0], org[1]))  # Tuple of (original image, adversarial image, label)

    # Prepare DataLoader for evaluation
    testloader = torch.utils.data.DataLoader(eval_set, batch_size=4, shuffle=False, num_workers=2)

    # Initialize counters
    org_tp, org_tp_5, adv_tp, adv_tp_5, counter, diff = 0., 0., 0., 0., 0., 0.

    # Evaluate the model
    for imgs, adv_imgs, labels in tqdm.tqdm(testloader, desc="Validating..."):
        imgs = imgs.to('cuda:0')
        adv_imgs = adv_imgs.to('cuda:0')
        labels = labels.to('cuda:0')

        with torch.no_grad():
            org_preds = model(imgs)
            adv_preds = model(adv_imgs)

        # Calculate the differences
        diff += torch.sum(torch.abs(adv_imgs - imgs)) / (3 * 32 * 32)  # Normalize by the number of elements per image

        # Calculate accuracies
        org_tp += (org_preds.argmax(dim=-1) == labels).sum()
        org_tp_5 += (org_preds.topk(5, dim=-1)[1] == labels[..., None]).any(dim=-1).sum()
        adv_tp += (adv_preds.argmax(dim=-1) == labels).sum()
        adv_tp_5 += (adv_preds.topk(5, dim=-1)[1] == labels[..., None]).any(dim=-1).sum()
        counter += org_preds.shape[0]

    # Calculate final accuracies
    org_acc = org_tp.float().item() / counter
    org_top5 = org_tp_5.float().item() / counter
    adv_acc = adv_tp.float().item() / counter
    adv_top5 = adv_tp_5.float().item() / counter

    # Determine if the results are satisfactory
    result = "correct" if org_acc - adv_acc >= 0.2 and org_top5 - adv_top5 >= 0.15 else "need to improve" # and diff / counter <= 0.1

    # Print the results
    print(f"Top-1 error on original samples: {(100.0 * (1 - org_acc)):4.2f}%; \n Top-1 error on adversarial samples: {(100.0 * (1 - adv_acc)):4.2f}%")
    print(f"Top-5 error on original samples: {(100.0 * (1 - org_top5)):4.2f}%; \n Top-5 error on adversarial samples: {(100.0 * (1 - adv_top5)):4.2f}%")
    print(f'org_acc - adv_acc: {(100.0 * (org_acc - adv_acc)):4.2f}% \n org_top5 - adv_top5: {(100.0 * (org_top5 - adv_top5)):4.2f}%')
    print(f'diff / counter: {(diff / counter)}')
    print(result)

    # Warn if the average perturbation per image is too large
    if (diff / counter > 0.1):
        print("epsilon > 0.1")

In [16]:
# Assuming the model is loaded and set to evaluation mode
self_check(model, 'fgsm.pkl')

Validating...: 100%|██████████| 25/25 [00:01<00:00, 22.53it/s]

Top-1 error on original samples: 18.00%; 
 Top-1 error on adversarial samples: 80.00%
Top-5 error on original samples: 3.00%; 
 Top-5 error on adversarial samples: 18.00%
org_acc - adv_acc: 62.00% 
 org_top5 - adv_top5: 15.00%
diff / counter: 0.19260919094085693
correct
epsilon > 0.1


In [17]:
# Assuming the model is loaded and set to evaluation mode
self_check(model, 'pgd.pkl')

Validating...: 100%|██████████| 25/25 [00:01<00:00, 22.72it/s]

Top-1 error on original samples: 18.00%; 
 Top-1 error on adversarial samples: 85.00%
Top-5 error on original samples: 3.00%; 
 Top-5 error on adversarial samples: 18.00%
org_acc - adv_acc: 67.00% 
 org_top5 - adv_top5: 15.00%
diff / counter: 0.4278186559677124
correct
epsilon > 0.1


## References:
1. [Adversarial attacks with FGSM (Fast Gradient Sign Method)](https://pyimagesearch.com/2021/03/01/adversarial-attacks-with-fgsm-fast-gradient-sign-method/)
2. [Adversarial example using FGSM](https://www.tensorflow.org/tutorials/generative/adversarial_fgsm)
3. [Adversarial Training for Free!](https://proceedings.neurips.cc/paper_files/paper/2019/file/7503cfacd12053d309b6bed5c89de212-Paper.pdf)
4. [Towards Deep Learning Models Resistant to Adversarial
Attacks](https://arxiv.org/pdf/1706.06083.pdf)